In [1]:
import tensorflow as tf
from mlff.data_prep import dataset_from_json

In [60]:
from itertools import combinations_with_replacement
from mlff.utils import distances_and_pair_types
import json

In [3]:
from mlpot.descriptors import DescriptorSet

In [5]:
ds = DescriptorSet(['Ni', 'Pt'], cutoff=7.0)
ds.add_Artrith_Kolpak_set()

/home/ralf/Development/py3.8/lib/python3.8/site-packages/mlpot/descriptors/descriptorset.py:106: UserWarning: The Behler atom-centered symmetry functions have been renamed and now follow the convention of J. Behler, JCP 134 074106
  warn('The Behler atom-centered symmetry functions have been renamed ' +
/home/ralf/Development/py3.8/lib/python3.8/site-packages/mlpot/descriptors/descriptorset.py:127: UserWarning: The Behler atom-centered symmetry functions have been renamed and now follow the convention of J. Behler, JCP 134 074106
  warn('The Behler atom-centered symmetry functions have been renamed ' +


In [152]:
def descriptor_dataset_from_json(path, descriptor_set,
                                 batch_size=None, buffer_size=None):
    with open(path, 'r') as fin:
        data = json.load(fin)

    if batch_size is None:
        batch_size = len(data['symbols'])
        buffer_size = batch_size
    buffer_size = buffer_size or 4*batch_size
    
    def py_fun(types, positions):
        Gs, dGs = ds.eval_with_derivatives_atomwise(types.numpy().ravel(), positions.numpy())
        return Gs, dGs

    def input_transform(inp):
        [Gs, dGs] = tf.py_function(py_fun, [inp['types'], tf.cast(inp['positions'], dtype=tf.float64)], [tf.float64, tf.float64])
        Gs = tf.cast(tf.ragged.stack(Gs), dtype=tf.float32)
        dGs = tf.cast(tf.ragged.stack(dGs), dtype=tf.float32)
        return dict(types=inp['types'], Gs=Gs, dGs=dGs)

    input_dataset = tf.data.Dataset.from_tensor_slices(
        {'types': tf.expand_dims(tf.ragged.constant(
                [[type_dict[t] for t in syms] for syms in data['symbols']]),
            axis=-1),
         'positions': tf.ragged.constant(data['positions'], ragged_rank=1)})

    input_dataset = input_dataset.map(
        input_transform, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    print(input_dataset)

    N = tf.constant([len(sym) for sym in data['symbols']], dtype=tf.float32)
    energy_per_atom = tf.constant(data['e_dft_bond'], dtype=tf.float32)/N
    output_dataset = tf.data.Dataset.from_tensor_slices(
        (energy_per_atom,
         tf.ragged.constant(data['forces_dft'], ragged_rank=1)))

    input_dataset = input_dataset.apply(
        tf.data.experimental.dense_to_ragged_batch(batch_size=batch_size))
    output_dataset = output_dataset.batch(batch_size=batch_size)

    dataset = tf.data.Dataset.zip((input_dataset, output_dataset))

    dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [153]:
type_dict = {'Ni': 0, 'Pt': 1}
dataset_val = descriptor_dataset_from_json('../data/val_data.json', ds, batch_size=1)

<ParallelMapDataset shapes: {types: (None, 1), Gs: <unknown>, dGs: <unknown>}, types: {types: tf.int32, Gs: tf.float32, dGs: tf.float32}>


In [150]:
inp, (ref_E, ref_forces) = list(dataset_val.take(1))[0]

In [151]:
inp.keys()

dict_keys(['types', 'Gs', 'dGs'])

In [55]:
%time Gs, dGs = ds.eval_with_derivatives_atomwise(inp['types'][0].numpy().ravel(), tf.cast(inp['positions'][0], tf.float64).numpy())
%time Gs = tf.cast(tf.ragged.stack(Gs), dtype=tf.float32)
%time dGs = tf.cast(tf.ragged.stack(dGs), dtype=tf.float32)

CPU times: user 6.45 s, sys: 0 ns, total: 6.45 s
Wall time: 1.71 s
CPU times: user 6.97 ms, sys: 0 ns, total: 6.97 ms
Wall time: 4.73 ms
CPU times: user 151 ms, sys: 7.84 ms, total: 159 ms
Wall time: 135 ms


In [31]:
class BehlerParrinello(tf.keras.Model):
    
    def __init__(self, atom_types, cutoff=7.0, build_forces=False):
        super().__init__()
        self.atom_types = sorted(atom_types)
        self.atom_pair_types = []
        for (t1, t2) in combinations_with_replacement(self.atom_types, 2):
            self.atom_pair_types.append(''.join([t1, t2]))
        self.build_forces = build_forces
        self.cutoff = cutoff
        
        types = tf.keras.Input(shape=(None, 1), ragged=True, dtype=tf.int32)
        positions = tf.keras.Input(shape=(None, 3), ragged=True)
        inputs = {'types': types, 'positions': positions}
        
    def call(self, inputs):
        types = inputs['types']
        positions = inputs['positions']
        distances, pair_types, dr_dx = tf.map_fn(
            lambda x: distances_and_pair_types(
                x[0], x[1], len(self.atom_types), self.cutoff),
            (positions, types),
            fn_output_signature=(tf.RaggedTensorSpec(
                                    shape=[None, None, 1], ragged_rank=1,
                                    dtype=positions.dtype),
                                 tf.RaggedTensorSpec(
                                    shape=[None, None, 1], ragged_rank=1,
                                    dtype=types.dtype),
                                 tf.RaggedTensorSpec(
                                    shape=[None, None, None, 3],
                                    ragged_rank=2, dtype=positions.dtype))
            )

        if self.build_forces:
            return self.main_body_with_forces(types, distances,
                                              pair_types, dr_dx)
        return self.main_body_no_forces(types, distances, pair_types)
    
    @tf.function
    def main_body_no_forces(self, types, distances, pair_types):
        """Calculates the energy per atom by calling body_partition_stitch()
        """
        energy = self.body_partition_stitch(types, distances, pair_types)
        # energy = self.body_gather_scatter(types, distances, pair_types)
        number_of_atoms = tf.cast(types.row_lengths(), energy.dtype,
                                  name='number_of_atoms')
        energy_per_atom = tf.divide(
            energy, tf.expand_dims(number_of_atoms, axis=-1),
            name='energy_per_atom')

        return energy_per_atom
    
    @tf.function
    def body_partition_stitch(self, types, distances, pair_types):
        """Returns the total energy """
        # Save indices for later stitching
        pair_type_indices = tf.dynamic_partition(
                tf.expand_dims(tf.range(tf.size(distances)), -1),
                pair_types.flat_values, len(self.atom_pair_types),
                name='pair_type_indices')
        # Partition distances according to the pair_type. Gives
        # a list of (None,) tensors of length len(atom_pair_types)
        partitioned_r = tf.dynamic_partition(
            distances.flat_values * 1, pair_types.flat_values,
            len(self.atom_pair_types), name='partitioned_r')
        print(partitioned_r)
        
        # Apply radial descriptors according to pair_type. Yields
        # 
        descriptors = [self.radial_descriptors[t](part)
         for t, part in zip(self.atom_pair_types, partitioned_r)]

In [32]:
model = BehlerParrinello(['Ni', 'Pt'])

In [33]:
inp, (ref_E, ref_forces) = list(dataset_val.take(1))[0]

In [34]:
model(inp)

[<tf.Tensor 'partitioned_r:0' shape=(None,) dtype=float32>, <tf.Tensor 'partitioned_r:1' shape=(None,) dtype=float32>, <tf.Tensor 'partitioned_r:2' shape=(None,) dtype=float32>]


AttributeError: in user code:

    <ipython-input-26-9fe9a6cb48e5>:45 main_body_no_forces  *
        number_of_atoms = tf.cast(types.row_lengths(), energy.dtype,

    AttributeError: 'Operation' object has no attribute 'dtype'


In [47]:
positions = inp['positions']
types = inp['types']

distances, pair_types, dr_dx = tf.map_fn(
            lambda x: distances_and_pair_types(
                x[0], x[1], 2, 7.0),
            (positions, types),
            fn_output_signature=(tf.RaggedTensorSpec(
                                    shape=[None, None, 1], ragged_rank=1,
                                    dtype=positions.dtype),
                                 tf.RaggedTensorSpec(
                                    shape=[None, None, 1], ragged_rank=1,
                                    dtype=types.dtype),
                                 tf.RaggedTensorSpec(
                                    shape=[None, None, None, 3],
                                    ragged_rank=2, dtype=positions.dtype))
            )

In [49]:
distances.shape

TensorShape([4, None, None, 1])

In [53]:
partitioned_r = tf.dynamic_partition(
            distances.flat_values * 1, pair_types.flat_values,
            3, name='partitioned_r')

In [59]:
positions.shape

TensorShape([4, None, 3])

In [58]:
positions.nested_row_lengths()

(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([147, 147,  55, 147])>,)

In [61]:
pair_type_indices = tf.dynamic_partition(
                tf.expand_dims(tf.range(tf.size(distances)), -1),
                pair_types.flat_values, 3,
                name='pair_type_indices')

In [95]:
tf.dynamic_partition(distances, tf.expand_dims(types, -1), 2)

InvalidArgumentError: data and partitions have incompatible ragged shapes
Condition x == y did not hold.
Indices of first 3 different values:
[[1]
 [2]
 [3]]
Corresponding x values:
[ 46  88 148]
Corresponding y values:
[1 2 3]
First 3 elements of x:
[ 0 46 88]
First 3 elements of y:
[0 1 2]

In [82]:
partitioned_r = tf.dynamic_partition(
            distances.flat_values, pair_types.flat_values,
            3, name='partitioned_r')

In [86]:
descriptors = [tf.expand_dims(partitioned_r[0], axis=-1) * tf.ones((1, 5)),
               tf.expand_dims(partitioned_r[1], axis=-1) * tf.ones((1, 7)),
               tf.expand_dims(partitioned_r[2], axis=-1) * tf.ones((1, 5))]

In [89]:
pair_type_indices

[<tf.Tensor: shape=(5698,), dtype=int32, numpy=array([ 4192,  4193,  4194, ..., 27019, 27020, 27021], dtype=int32)>,
 <tf.Tensor: shape=(12572,), dtype=int32, numpy=array([   23,    24,    25, ..., 27004, 27005, 27006], dtype=int32)>,
 <tf.Tensor: shape=(8752,), dtype=int32, numpy=array([    0,     1,     2, ..., 24906, 24907, 24908], dtype=int32)>]

In [87]:
stitched_descriptors = tf.dynamic_stitch(pair_type_indices, descriptors)

InvalidArgumentError: Need data[0].shape[1:] = data[1].shape[1:], got data[0].shape = [5698,5], data[1].shape = [12572,7], indices[0].shape = [5698], indices[1].shape = [12572] [Op:DynamicStitch]

In [88]:
tf.RaggedTensor.from_nested_row_splits(stitched_descriptors, distances.nested_row_splits).shape

TensorShape([4, None, None, 5])